In [1]:
import os
import tensorflow as tf
import numpy as np 

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.chdir('/home/aditya/Downloads/TensorflowDatasets/train')

### Extracting Filenames and Classes

In [3]:
classes=[]
images=[]
for i in os.listdir('/home/aditya/Downloads/TensorflowDatasets/train'):
    filename=tf.read_file(i)
    images.append(filename)
    if i[0]=='c':
        classes.append(0)
    else:
        classes.append(1)

### One_hot encoding of Classes

In [4]:
y_true=tf.one_hot(classes,2)

In [5]:
images[1]

<tf.Tensor 'ReadFile_1:0' shape=() dtype=string>

### Creating Training(24000 images) and Validation Datasets(1000 images)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((images[:24000], y_true[:24000]))
validation_dataset = tf.data.Dataset.from_tensor_slices((images[24000:25000], y_true[24000:25000]))

In [7]:
len(images)

25000

In [8]:
dataset

<TensorSliceDataset shapes: ((), (2,)), types: (tf.string, tf.float32)>

### Getting Images from Filenames

In [9]:
def parse_function(images, y_true):
    image = tf.image.decode_jpeg(images, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_images(image, [64, 64])
    return image, y_true

In [10]:
dataset = dataset.map(parse_function, num_parallel_calls=4)
validation_dataset = validation_dataset.map(parse_function, num_parallel_calls=4)

### Augmentation(Flip_up_down,Flip_Left_Right,Brightness,Saturation) with 50% chance. 

In [11]:
def train_preprocess(image, y_true):
    a=np.random.randint(1,5)
    b=np.random.randint(1,3)
    if b==1:
        image=image
    else:
        if a==1:
            image = tf.image.random_flip_left_right(image)
        elif a==2:
            image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
        elif a==3:
            image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        else:
            image = tf.image.random_flip_up_down(image)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image,y_true

In [12]:
dataset = dataset.map(train_preprocess, num_parallel_calls=2)

In [13]:
dataset

<ParallelMapDataset shapes: ((64, 64, 3), (2,)), types: (tf.float32, tf.float32)>

### Shuffling Dataset

In [14]:
dataset=dataset.shuffle(24000)

### Batchsize=16

In [15]:
dataset=dataset.batch(16)

In [16]:
dataset

<BatchDataset shapes: ((?, 64, 64, 3), (?, 2)), types: (tf.float32, tf.float32)>

### Reinitializable Iterator

In [17]:
iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)
X,y = iterator.get_next()

In [18]:
y

<tf.Tensor 'IteratorGetNext:1' shape=(?, 2) dtype=float32>

In [19]:
os.getcwd()

'/home/aditya/Downloads/TensorflowDatasets/train'

In [20]:
dataset

<BatchDataset shapes: ((?, 64, 64, 3), (?, 2)), types: (tf.float32, tf.float32)>

In [21]:
X=tf.reshape(X,[-1,64,64,3])

### VGG 1st Layer + 1 Fully Connected into Prediction

In [22]:
conv1=tf.layers.conv2d(X,64,3)
conv2=tf.layers.conv2d(conv1,64,3)
maxpool1=tf.layers.max_pooling2d(conv2,[2,2],[2,2])
flat=tf.layers.flatten(maxpool1)
dense1=tf.layers.dense(flat,1024)
y_pred=tf.layers.dense(dense1,2)

In [23]:
y_pred

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 2) dtype=float32>

In [24]:
y_true

<tf.Tensor 'one_hot:0' shape=(25000, 2) dtype=float32>

### Initializing Iterator

In [25]:
validation_dataset=validation_dataset.batch(16)
training_init_op = iterator.make_initializer(dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

### Training and Loss(Softmax)

In [26]:
loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))
train_op=tf.train.AdamOptimizer().minimize(loss_op) 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Accuracy 

In [27]:
prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

### Session

In [28]:
 with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for j in range(2):
        sess.run(training_init_op)
        for i in range(1500):
            print(sess.run([train_op,loss_op,accuracy]))
        sess.run(validation_init_op)
        for k in range(62):
            print('Validation Accuracy Batch[k]',sess.run(accuracy))

[None, 0.652299, 0.625]
[None, 18.879856, 0.625]
[None, 24.499725, 0.75]
[None, 55.699898, 0.1875]
[None, 27.556019, 0.625]
[None, 52.85239, 0.4375]
[None, 30.83721, 0.4375]
[None, 4.30204, 0.8125]
[None, 39.00325, 0.4375]
[None, 16.332977, 0.625]
[None, 4.8666897, 0.6875]
[None, 7.8049035, 0.5625]
[None, 16.556557, 0.4375]
[None, 14.575241, 0.375]
[None, 4.837511, 0.25]
[None, 3.9147763, 0.5625]
[None, 7.3022146, 0.625]
[None, 10.8420105, 0.4375]
[None, 5.923153, 0.375]
[None, 3.0641026, 0.25]
[None, 8.173194, 0.5]
[None, 7.1879907, 0.4375]
[None, 3.5164924, 0.625]
[None, 2.627275, 0.4375]
[None, 2.5434594, 0.4375]
[None, 5.471014, 0.4375]
[None, 2.7033477, 0.625]
[None, 1.6512015, 0.6875]
[None, 1.434765, 0.6875]
[None, 1.6690025, 0.5625]
[None, 1.4027407, 0.6875]
[None, 3.8002417, 0.5]
[None, 4.05271, 0.4375]
[None, 0.5286528, 0.6875]
[None, 1.2793518, 0.5]
[None, 0.8441005, 0.75]
[None, 1.9205561, 0.5]
[None, 1.1692555, 0.6875]
[None, 1.8199797, 0.5625]
[None, 1.4857563, 0.6875]
[N

[None, 0.68975574, 0.625]
[None, 0.6661285, 0.6875]
[None, 0.62925804, 0.6875]
[None, 0.5753586, 0.8125]
[None, 0.68269813, 0.5625]
[None, 0.70301294, 0.5625]
[None, 0.5137451, 0.8125]
[None, 0.47623625, 0.75]
[None, 0.67795986, 0.5625]
[None, 0.71468747, 0.625]
[None, 0.5536039, 0.6875]
[None, 0.612911, 0.625]
[None, 0.6077839, 0.5]
[None, 0.62696356, 0.6875]
[None, 0.6082102, 0.5625]
[None, 0.6988651, 0.6875]
[None, 0.50809675, 0.75]
[None, 0.69804573, 0.4375]
[None, 0.5534247, 0.8125]
[None, 0.67147964, 0.75]
[None, 0.50920606, 0.6875]
[None, 0.7908033, 0.5625]
[None, 0.74744743, 0.625]
[None, 0.74339306, 0.5]
[None, 0.56821454, 0.8125]
[None, 0.60985833, 0.625]
[None, 0.48806995, 0.75]
[None, 0.60751355, 0.6875]
[None, 0.53628266, 0.75]
[None, 0.40916193, 0.9375]
[None, 0.75060976, 0.625]
[None, 0.7008939, 0.375]
[None, 0.83244085, 0.5]
[None, 0.5314772, 0.8125]
[None, 0.5563312, 0.6875]
[None, 0.57422864, 0.75]
[None, 0.78179944, 0.5]
[None, 0.6116382, 0.6875]
[None, 0.62429875, 0

[None, 0.54950166, 0.75]
[None, 0.612726, 0.6875]
[None, 0.64039195, 0.625]
[None, 0.5324223, 0.6875]
[None, 0.5220242, 0.6875]
[None, 0.7030233, 0.5625]
[None, 0.7249775, 0.6875]
[None, 0.64602, 0.625]
[None, 0.69596726, 0.5625]
[None, 0.76136506, 0.5]
[None, 0.8327389, 0.3125]
[None, 0.56990564, 0.6875]
[None, 0.5575366, 0.75]
[None, 0.50503886, 0.875]
[None, 0.62551725, 0.5625]
[None, 0.78307766, 0.625]
[None, 0.4878283, 0.8125]
[None, 0.5406964, 0.6875]
[None, 0.62156975, 0.5625]
[None, 0.5122014, 0.75]
[None, 0.74007905, 0.5]
[None, 0.41342783, 0.875]
[None, 0.5859047, 0.75]
[None, 0.5957954, 0.75]
[None, 0.5982039, 0.625]
[None, 0.5724468, 0.8125]
[None, 0.54463375, 0.6875]
[None, 0.5326312, 0.6875]
[None, 0.61267775, 0.75]
[None, 0.5533211, 0.75]
[None, 0.6529864, 0.625]
[None, 0.6155627, 0.6875]
[None, 0.841785, 0.5]
[None, 0.67455846, 0.5625]
[None, 0.5910835, 0.6875]
[None, 0.65660775, 0.5625]
[None, 0.58814, 0.8125]
[None, 0.590422, 0.75]
[None, 0.42665815, 0.8125]
[None, 0.

[None, 0.54986876, 0.8125]
[None, 0.4776801, 0.75]
[None, 0.5418868, 0.75]
[None, 0.66050035, 0.5]
[None, 0.60761285, 0.8125]
[None, 0.5835358, 0.8125]
[None, 0.54356694, 0.8125]
[None, 0.7086508, 0.5]
[None, 0.42677245, 0.875]
[None, 0.544088, 0.625]
[None, 0.61645436, 0.625]
[None, 0.62110937, 0.6875]
[None, 0.6333653, 0.75]
[None, 0.51323676, 0.75]
[None, 0.46792358, 0.8125]
[None, 0.58386827, 0.75]
[None, 0.5685124, 0.625]
[None, 0.87056184, 0.5]
[None, 0.58242786, 0.75]
[None, 0.7363517, 0.5]
[None, 0.52288395, 0.75]
[None, 0.66158295, 0.625]
[None, 0.6020187, 0.625]
[None, 0.82810247, 0.625]
[None, 0.56524396, 0.75]
[None, 0.7129234, 0.625]
[None, 0.7234112, 0.625]
[None, 0.59791756, 0.625]
[None, 0.53754765, 0.75]
[None, 0.47926706, 0.875]
[None, 0.5834097, 0.75]
[None, 0.59405196, 0.625]
[None, 0.5268713, 0.75]
[None, 0.5802431, 0.6875]
[None, 0.6128196, 0.625]
[None, 0.6009244, 0.75]
[None, 0.5280256, 0.75]
[None, 0.54487056, 0.75]
[None, 0.6109425, 0.625]
[None, 0.5656457, 0.

[None, 0.6253275, 0.625]
[None, 0.3872873, 0.9375]
[None, 0.6839497, 0.625]
[None, 0.7291354, 0.5625]
[None, 0.74250174, 0.625]
[None, 0.46760058, 0.875]
[None, 0.571074, 0.6875]
[None, 0.46824044, 0.875]
[None, 0.5177926, 0.8125]
[None, 0.4271983, 0.9375]
[None, 0.42980623, 0.875]
[None, 0.49899316, 0.6875]
[None, 0.62700343, 0.5625]
[None, 0.6663847, 0.5625]
[None, 0.43389997, 0.6875]
[None, 0.46452188, 0.75]
[None, 0.7891866, 0.75]
[None, 0.6811226, 0.6875]
[None, 0.5476016, 0.625]
[None, 0.6403936, 0.625]
[None, 0.4567355, 0.75]
[None, 0.6876884, 0.4375]
[None, 0.68446815, 0.5625]
[None, 0.44265935, 0.8125]
[None, 0.54435617, 0.625]
[None, 0.424177, 0.8125]
[None, 0.5130819, 0.75]
[None, 0.6435223, 0.6875]
[None, 0.5941181, 0.6875]
[None, 0.7323415, 0.6875]
[None, 0.67278135, 0.625]
[None, 0.4817486, 0.75]
[None, 0.56597596, 0.5625]
[None, 0.60630614, 0.75]
[None, 0.44674382, 0.75]
[None, 0.53710616, 0.625]
[None, 0.25429305, 1.0]
[None, 0.4742089, 0.875]
[None, 0.55972254, 0.625]


[None, 0.5450593, 0.75]
[None, 0.4524946, 0.6875]
[None, 0.3981352, 0.875]
[None, 0.35342026, 0.8125]
[None, 0.33478236, 0.8125]
[None, 0.30657637, 0.875]
[None, 0.3723778, 0.8125]
[None, 0.8060482, 0.6875]
[None, 0.39044288, 0.8125]
[None, 0.40847185, 0.875]
[None, 0.5530416, 0.6875]
[None, 0.74141604, 0.5625]
[None, 0.46426833, 0.6875]
[None, 0.3961891, 0.8125]
[None, 0.6619035, 0.6875]
[None, 0.30446684, 0.875]
[None, 0.39385492, 0.875]
[None, 0.4906699, 0.875]
[None, 0.68176854, 0.5]
[None, 0.60793483, 0.625]
[None, 0.58721745, 0.75]
[None, 0.54025674, 0.8125]
[None, 0.5810962, 0.8125]
[None, 0.5121922, 0.75]
[None, 0.42557472, 0.8125]
[None, 0.3483039, 0.75]
[None, 0.43655056, 0.875]
[None, 0.6670056, 0.5625]
[None, 0.55698323, 0.6875]
[None, 0.40876955, 0.8125]
[None, 0.4983353, 0.8125]
[None, 0.6120959, 0.625]
[None, 0.5716338, 0.75]
[None, 0.5162061, 0.6875]
[None, 0.45220226, 0.75]
[None, 0.4482426, 0.75]
[None, 0.74166983, 0.6875]
[None, 0.525509, 0.6875]
[None, 0.40802148, 0

[None, 0.83985925, 0.5625]
[None, 0.3483695, 0.875]
[None, 0.38330376, 0.8125]
[None, 1.0978218, 0.375]
[None, 0.43831694, 0.75]
[None, 0.7640636, 0.6875]
[None, 0.5730227, 0.625]
[None, 0.9014513, 0.5625]
[None, 0.52685386, 0.625]
[None, 0.39227638, 0.8125]
[None, 0.5718297, 0.625]
[None, 0.36826873, 0.8125]
[None, 0.6326871, 0.6875]
[None, 0.6865245, 0.5625]
[None, 0.5839201, 0.6875]
[None, 0.45797998, 0.8125]
[None, 0.3358159, 1.0]
[None, 0.6267048, 0.6875]
[None, 0.6466551, 0.5625]
[None, 0.30025667, 0.875]
[None, 0.3965628, 0.875]
[None, 0.4348647, 0.875]
[None, 0.53643143, 0.6875]
[None, 0.42778513, 0.875]
[None, 0.75954664, 0.625]
[None, 0.30093193, 0.875]
[None, 0.48905474, 0.6875]
[None, 0.80458695, 0.6875]
[None, 0.60396165, 0.6875]
[None, 0.92747855, 0.5625]
[None, 0.5398376, 0.6875]
[None, 0.5895963, 0.6875]
[None, 0.6012373, 0.8125]
[None, 0.6377326, 0.625]
[None, 0.4581766, 0.8125]
[None, 0.6184372, 0.6875]
[None, 0.44028968, 0.75]
[None, 0.6574062, 0.5625]
[None, 0.39817

[None, 0.55742836, 0.5625]
[None, 0.53223276, 0.75]
[None, 0.40788463, 0.8125]
[None, 0.6818248, 0.625]
[None, 0.26006058, 0.9375]
[None, 0.45327485, 0.8125]
[None, 0.7773348, 0.625]
[None, 0.9365732, 0.4375]
[None, 0.3643027, 0.875]
[None, 0.60719687, 0.625]
[None, 0.60927266, 0.75]
[None, 0.5025216, 0.625]
[None, 0.46106404, 0.6875]
[None, 0.42218143, 0.75]
[None, 0.6251168, 0.75]
[None, 0.51443684, 0.75]
[None, 0.5209487, 0.8125]
[None, 0.8033829, 0.5625]
[None, 0.54710126, 0.75]
[None, 0.46412992, 0.75]
[None, 0.58434147, 0.8125]
[None, 0.9794262, 0.5625]
[None, 0.47889388, 0.6875]
[None, 0.3990758, 0.9375]
[None, 0.5047023, 0.75]
[None, 0.6713371, 0.5625]
[None, 0.5845629, 0.625]
[None, 0.43138227, 0.8125]
[None, 0.65956503, 0.625]
[None, 0.47570822, 0.75]
[None, 0.7739281, 0.5]
[None, 0.43233448, 0.8125]
[None, 0.5511549, 0.6875]
[None, 0.29910138, 1.0]
[None, 0.41823715, 0.9375]
[None, 0.53114724, 0.8125]
[None, 0.62358403, 0.625]
[None, 0.5353399, 0.6875]
[None, 0.7183659, 0.62

[None, 0.59169865, 0.6875]
[None, 0.60624087, 0.625]
[None, 0.33760813, 0.875]
[None, 0.77163804, 0.5625]
[None, 0.5612789, 0.8125]
[None, 0.4632359, 0.8125]
[None, 0.75411963, 0.625]
[None, 0.39377323, 0.875]
[None, 0.58916783, 0.5625]
[None, 0.3915451, 0.875]
[None, 0.5297909, 0.75]
[None, 0.5781097, 0.6875]
[None, 0.61292064, 0.625]
[None, 0.5713089, 0.8125]
[None, 0.70867443, 0.625]
[None, 0.5801786, 0.6875]
[None, 0.25899816, 0.875]
[None, 0.5556142, 0.625]
[None, 0.5348903, 0.5625]
[None, 0.65899074, 0.5625]
[None, 0.44303215, 0.75]
[None, 0.5734497, 0.625]
[None, 0.71631145, 0.6875]
[None, 0.5437937, 0.625]
[None, 0.4677071, 0.875]
[None, 0.28110167, 1.0]
[None, 0.39098758, 0.9375]
[None, 0.7265748, 0.625]
[None, 0.3981307, 0.875]
[None, 0.39677516, 0.875]
[None, 0.53396034, 0.875]
[None, 0.35625026, 0.9375]
[None, 0.37307456, 0.875]
[None, 0.4995089, 0.6875]
[None, 0.66565037, 0.75]
[None, 0.4419638, 0.75]
[None, 0.32036033, 0.875]
[None, 0.60704094, 0.6875]
[None, 0.557955, 0.

[None, 1.1471066, 0.5]
[None, 0.54598486, 0.75]
[None, 0.4579749, 0.625]
[None, 0.89896226, 0.625]
[None, 0.5211829, 0.75]
[None, 0.739941, 0.625]
[None, 0.3356201, 1.0]
[None, 0.5034162, 0.6875]
[None, 0.38331562, 0.9375]
[None, 0.5226792, 0.6875]
[None, 0.33758056, 0.8125]
[None, 0.4005674, 0.8125]
[None, 0.71718025, 0.75]
[None, 0.3627491, 0.875]
[None, 0.57683736, 0.75]
[None, 0.28347155, 0.9375]
[None, 0.77005804, 0.5]
[None, 1.0102394, 0.5625]
[None, 0.64466286, 0.6875]
[None, 0.70785034, 0.75]
[None, 0.4903461, 0.75]
[None, 0.6554514, 0.6875]
[None, 0.5306539, 0.8125]
[None, 0.520514, 0.6875]
[None, 0.42481655, 0.875]
[None, 0.6916801, 0.625]
[None, 0.46250385, 0.75]
[None, 0.4326919, 0.8125]
[None, 0.61762005, 0.6875]
[None, 0.32823092, 0.875]
[None, 0.5461293, 0.75]
[None, 0.8486949, 0.5]
[None, 0.5911602, 0.6875]
[None, 0.39179996, 0.8125]
[None, 0.6370673, 0.6875]
[None, 0.85999453, 0.375]
[None, 0.49859446, 0.75]
[None, 0.47322547, 0.6875]
[None, 0.483387, 0.6875]
[None, 0.

In [29]:
X.shape

TensorShape([Dimension(None), Dimension(64), Dimension(64), Dimension(3)])

In [30]:
y.shape

TensorShape([Dimension(None), Dimension(2)])

In [31]:
y_pred.shape

TensorShape([Dimension(None), Dimension(2)])